# General Preparation

In [4]:
from datasets import load_dataset, load_from_disk
import os
from pathlib import Path
from typing import Dict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    DataCollatorForSeq2Seq,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Seq2SeqTrainingArguments,
    Trainer,
    Seq2SeqTrainer,
    PreTrainedTokenizerFast,
)
from functools import partial
from transformers.models.qwen2.tokenization_qwen2_fast import Qwen2TokenizerFast
from transformers.models.qwen2.modeling_qwen2 import Qwen2ForCausalLM, Qwen2ForSequenceClassification
from peft import PrefixTuningConfig, get_peft_model, TaskType, LoraConfig
from peft.peft_model import PeftModelForCausalLM, PeftModelForSequenceClassification
from rouge_score import rouge_scorer
import numpy as np
from transformers import GenerationConfig
import torch
import json

## Hyper Parameters for Training

In [5]:
batch_size = 1
accumulation_steps = 8
learning_rate = 2e-4
epochs = 3
warmup_ratio = 0.05
weight_decay = 0.1
save_total_limit=2
logging_steps = 0.1


smoke_run = False

## Directory Settings

In [6]:
# for autodl environment, use this line
file_dir = Path(os.getcwd())

# for local environment, use this line
# file_dir = Path(os.getcwd(), "sft")

file_dir

PosixPath('/root/llm_adv_qa/sft')

In [7]:
# for autodl environment, use this line
cache_dir='/root/autodl-tmp'

# for local environment, use this line
# cache_dir=Path.home().as_posix()

cache_dir

'/root/autodl-tmp'

In [8]:
model_id = "Qwen/Qwen2.5-3B-Instruct"
model_path = str(
        Path(cache_dir, ".cache/modelscope/hub", model_id)
    )
model_path

'/root/autodl-tmp/.cache/modelscope/hub/Qwen/Qwen2.5-3B-Instruct'

In [9]:
os.path.exists(model_path)

True

In [10]:
model_output_dir = Path(
        file_dir.parent, "resources", "sft_models", "nl2sql"
    )
model_output_dir.mkdir(parents=True, exist_ok=True)
model_output_dir

PosixPath('/root/llm_adv_qa/resources/sft_models/nl2sql')

In [11]:
best_model_output_dir = Path(model_output_dir, "best")
best_model_output_dir

PosixPath('/root/llm_adv_qa/resources/sft_models/nl2sql/best')

# Data Preprocessing

In [9]:
train_file = Path(file_dir.parent, "resources", "dataset", "nl2sql", "train.jsonl")
test_file = Path(file_dir.parent, "resources", "dataset", "nl2sql", "test.jsonl")
dataset = load_dataset(
    "json", data_files={"train": train_file.as_posix(), "test": test_file.as_posix()}
).shuffle(seed=1234)

dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'prompt', 'label'],
        num_rows: 1026
    })
    test: Dataset({
        features: ['question', 'prompt', 'label'],
        num_rows: 115
    })
})

In [10]:
len(dataset['train'])

1026

In [11]:
logging_step_per_epoch = int(round(logging_steps * (len(dataset['train']) // (accumulation_steps * batch_size)), 0))
logging_step_per_epoch

13

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="left")
type(tokenizer)

transformers.models.qwen2.tokenization_qwen2_fast.Qwen2TokenizerFast

In [13]:
def preprocess_data(
    example,
    tokenizer: PreTrainedTokenizerFast,
) -> Dict:
    """
    example:
      {
          "prompt": "xxxx",
          "label": "xxx",
          "question": "xxx"
      },

    return:
        {"input_ids:[], attention_mask:[], labels:[]}

    input_ids格式: `X Y <eos>...`
    labels格式: `<ignore> ... <ignore> Y <eos>...`
    这里不做padding, 在后面让DataCollatorForSeq2Seq在训练时动态根据longest策略来做padding, 
    这样可以省去很多无用的padding空间, 提高训练效率
    """

    x: str = example["prompt"]
    y: str = example["label"]

    # 用模型适配的ChatML格式组成输入
    messages = [{"role": "user", "content": x}]
    x = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    x_ids = tokenizer.encode(x, truncation=True)
    y_ids = tokenizer.encode(y, truncation=True)
    eos = tokenizer.eos_token_id
    pad = tokenizer.pad_token_id
    
    input_ids = x_ids + y_ids +[eos]
    labels = [-100] * len(x_ids) + y_ids + [eos]
    
    attention_mask = [1] * len(x_ids) + [0] * len(y_ids +[eos])
    # return {"input_ids": input_ids, "labels": labels}
    return {"input_ids": input_ids, "labels": labels, "attention_mask": attention_mask}

In [14]:
preprocess_func = partial(preprocess_data, tokenizer=tokenizer)
original_cols = ["prompt", "label", "question"]

In [15]:
train_set = dataset["train"].map(
        partial(preprocess_func, tokenizer=tokenizer),
        remove_columns=original_cols,
)

validation_set = dataset["test"].map(
        partial(preprocess_func, tokenizer=tokenizer), remove_columns=original_cols
)

In [16]:
train_set.to_pandas()

,input_ids,labels,attention_mask
0,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...
1021,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1022,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1023,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1024,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [17]:
validation_set.to_pandas()

,input_ids,labels,attention_mask
0,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...
110,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
111,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
112,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
113,"[151644, 8948, 198, 2610, 525, 1207, 16948, 11...","[-100, -100, -100, -100, -100, -100, -100, -10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


## Have a Look at Processed Samples

In [18]:
tokenizer.decode(validation_set[1]['input_ids'])

"<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n\n# 任务\n你是一名Mysql数据库开发人员, 你精通Mysql数据库的sql语句编写, 你需要根据已知的表名、字段名和用户输入来编写sql代码.\n\n# 上下文\n## 已知表名\n- company_table\n\n## 已知字段名\n- 公司全称\n- 年份\n- 经营活动现金流入小计\n- 公司的中文简称\n- 固定资产\n- 应交税费\n- 应付职工薪酬\n- 未分配利润\n- 负债合计\n- 电子信箱\n- 资产总计\n- 无形资产\n- 货币资金\n- 资本公积\n- 利息收入\n- 营业收入\n- 营业外支出\n- 盈余公积\n- 营业利润\n- 营业外收入\n- 所得税费用\n- 其他收益\n- 现金及现金等价物净增加额\n- 净利润\n- 其他应收款\n- 营业成本\n- 综合收益总额\n- 流动资产合计\n- 应收账款\n- 预付款项\n- 其他应付款\n- 非流动资产合计\n- 基本每股收益\n- 购买商品\n- 接受劳务支付的现金\n- 应付账款\n- 流动负债合计\n- 利润总额\n- 管理费用\n- 其他流动资产\n- 递延所得税资产\n- 财务费用\n- 营业总收入\n- 非流动负债合计\n- 存货\n- 分配股利\n- 利润或偿付利息支付的现金\n- 稀释每股收益\n- 所有者权益合计\n- 营业总成本\n- 销售费用\n- 负债和所有者权益总计\n- 持续经营净利润\n- 信用减值损失\n- 财务人员\n- 销售人员\n- 投资收益\n- 行政人员\n- 技术人员\n- 利息费用\n- 生产人员\n- 研发费用\n- 资产减值损失\n- 递延收益\n- 其他非流动资产\n- 短期借款\n- 在职员工的数量合计\n\n# 要求\n- sql代码中的字段名必须是已知字段名,不得新增字段名\n\n# 示例\n- 输入:在上海注册的上市公司中,2019年谁的负债合计最高?金额是?\n  输出: ```select 公司全称, 负债合计 from company_table where 注册地址 L

In [19]:
test_labels = [id for id in validation_set[1]['labels'] if id != -100]
tokenizer.decode(test_labels)

"```select avg(税金及附加) from company_table where 年份 = '2020' and (注册地址 like '%北京市%' or 注册地址 like '%深圳%') and 税金及附加 is not null```<|im_end|>"

# Modeling

In [20]:
model: AutoModelForCausalLM = AutoModelForCausalLM.from_pretrained(model_path)
type(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

transformers.models.qwen2.modeling_qwen2.Qwen2ForCausalLM

In [21]:
def get_all_linear_layers(model):
    # Create a list to store the layer names
    layer_names = []
    
    # Recursively visit all modules and submodules
    for name, module in model.named_modules():
        # Check if the module is an instance of the specified layers
        if isinstance(module, (torch.nn.Linear)):

            layer_names.append('.'.join(name.split('.')[4:]).split('.')[0])
    name_map = {name: True for name in layer_names}
    
    return [name for name, _ in name_map.items() if name != '']

In [22]:
all_linear_layers = get_all_linear_layers(model)
all_linear_layers

['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']

In [23]:
# peft_config = PrefixTuningConfig(
#     task_type=TaskType.CAUSAL_LM, 
#     num_virtual_tokens=num_virtual_tokens, 
#     prefix_projection=False,
# )

peft_config = LoraConfig(
    r=8,  # Rank of LoRA matrices
    lora_alpha=32,  # Scaling factor
    target_modules=all_linear_layers,  # Target layers for LoRA
    lora_dropout=0.15,  # Dropout rate for LoRA layers
    bias="none",  # Optionally add bias terms
    task_type="CAUSAL_LM"  # Type of task
)


peft_config

LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules={'o_proj', 'k_proj', 'up_proj', 'v_proj', 'down_proj', 'gate_proj', 'q_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.15, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [24]:
peft_model: PeftModelForCausalLM = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 14,966,784 || all params: 3,100,905,472 || trainable%: 0.4827


# Metrics Function

In [25]:
from transformers import EvalPrediction
from metrics import BleuCalculator

def compute_metrics(eval_pred: EvalPrediction, tokenizer: PreTrainedTokenizerFast):
    predictions, labels, inputs = eval_pred
    labels = [[id for id in label if id != -100] for label in labels]
    predictions = [[id for id in pred if id != -100] for pred in predictions]
    inputs = [[id for id in input_ if id != -100] for input_ in inputs]
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_inputs = tokenizer.batch_decode(inputs, skip_special_tokens=True)

    score_dict = {}
    score_dict["bleu-4"] = []
    for prediction, label, input_, raw_pred in zip(decoded_preds, decoded_labels, decoded_inputs, predictions):
        new_text = prediction[len(input_):].replace("\n", "")
        bleu_score = BleuCalculator.calculate(new_text, label)
        score_dict["bleu-4"].append(bleu_score)
        print(f"new_text: ||{new_text}||")
        print(f"label: {label}")
        # print(f"pred: {prediction}")
        # print(f"input: {input_}")
        # print(f"raw pred: {raw_pred}")
        # print(f"raw pred len: {len(raw_pred)}")
        # print(f"new_text len: {len(new_text)}")
        # generated_keywords = new_text.split(",")
        # print(new_text)
    return {k: round(np.mean(v), 4) for k, v in score_dict.items()}


## A Simple Test of Our Eval Function

In [26]:
inputs = tokenizer("请生成sql:")['input_ids']
inputs = np.array(inputs)

predictions = tokenizer('请生成sql:select * from a where b = c' + tokenizer.eos_token)['input_ids']
predictions = np.array(predictions)

labels = tokenizer.encode('select * from a where b = e') + [-100] + [tokenizer.eos_token_id]
labels = np.array(labels)

ep = EvalPrediction([predictions], [labels], [inputs])

compute_metrics(ep, tokenizer)


new_text: ||select * from a where b = c||
label: select * from a where b = e


{'bleu-4': 0.8409}

In [27]:
metrics_func = partial(compute_metrics, tokenizer=tokenizer)

# Training

In [28]:
generation_config = GenerationConfig.from_pretrained(model_path)
generation_config.temperature = 1
generation_config.do_sample = False
generation_config.repetition_penalty = None
generation_config.top_k = None
generation_config.top_p = None
generation_config.max_length = 3000
generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "eos_token_id": [
    151645,
    151643
  ],
  "max_length": 3000,
  "pad_token_id": 151643,
  "repetition_penalty": null,
  "top_k": null,
  "top_p": null
}

In [29]:
# logging_steps = logging_step_per_epoch
# logging_steps

In [30]:
args = Seq2SeqTrainingArguments(
        output_dir=str(model_output_dir),
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=accumulation_steps,
        per_device_eval_batch_size=batch_size,
        eval_accumulation_steps=accumulation_steps,
        logging_steps=logging_steps,
        num_train_epochs=epochs,
        learning_rate=learning_rate,
        warmup_ratio=warmup_ratio,
        weight_decay=weight_decay,
        eval_strategy="steps",
        eval_steps=logging_steps,
        save_strategy="steps",
        save_total_limit=save_total_limit,
        save_steps=logging_steps,
        load_best_model_at_end=True,
        metric_for_best_model="bleu-4",
        predict_with_generate=True,
        include_for_metrics=["inputs, loss"],
        include_inputs_for_metrics=True,
        generation_config=generation_config,
    )

Using `include_inputs_for_metrics` is deprecated and will be removed in version 5 of 🤗 Transformers. Please use `include_for_metrics` list argument instead.


In [31]:
if smoke_run:
    validation_set = validation_set.select(range(8))
    train_set = train_set.select(range(8))

In [32]:
trainer = Seq2SeqTrainer(
        model=peft_model,
        args=args,
        processing_class=tokenizer,
        train_dataset=train_set,
        eval_dataset=validation_set,
        compute_metrics=metrics_func,
        data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding="longest", pad_to_multiple_of=8)
    )

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [33]:
from transformers.utils import logging
logging.set_verbosity_error() 

In [34]:
if smoke_run:
    trainer.evaluate(eval_dataset=validation_set)

In [35]:
if not smoke_run:
    trainer.train()
    trainer.save_model(output_dir=best_model_output_dir)
    trainer.save_state()

Step,Training Loss,Validation Loss,Bleu-4
39,6.702800,2.814280,0.422300
78,1.762200,1.145792,0.561200
117,0.900300,0.702266,0.552900
156,0.536700,0.455349,0.576300
195,0.381800,0.333746,0.602300
234,0.292200,0.235264,0.611100
273,0.209300,0.210761,0.586900
312,0.151600,0.183615,0.575600
351,0.124500,0.150755,0.561400


new_text: ||```select count(1) from company_table where 年份 = '2019' and 盈余公积 > 50000000 and 偿还债务支付的现金 > 1000000```||
label: ```select count(1) from company_table where 年份 = '2019'  and 盈余公积 is not null and 盈余公积 > 50000000 and 偿还债务支付的现金 is not null and 偿还债务支付的现金 > 1000000```
new_text: ||```select avg(税金及附加) from company_table where 年份 = '2020' and (注册地址 like '%北京市%' or 注册地址 like '%深圳%')```||
label: ```select avg(税金及附加) from company_table where 年份 = '2020' and (注册地址 like '%北京市%' or 注册地址 like '%深圳%') and 税金及附加 is not null```
new_text: ||```select sum(在职员工的数量) from company_table where 注册地址 like '%上海%' and 年份 = '2019' and 子公司在职员工的数量 is not null``` 以及  ```select sum(生产人员) from company_table where 注册地址 like '%上海%' and 年份 = '2019' and 生产人员 is not null```||
label: ```select sum(主要子公司在职员工的数量), sum(生产人员) from company_table where 年份 = '2019' and 注册地址 like '%上海市%'```
new_text: ||```select count(1) from company_table where 年份 = '2019' and 注册地址 like '%广州%' and 持续经营净利润 > 10000000 and 信用减值损失 > 10000000

# Push to ModelScope

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

AK = os.environ['MODEL_SCOPE_AK']

In [ ]:
from modelscope.hub.api import HubApi
from modelscope.hub.constants import Licenses, ModelVisibility

api = HubApi()
api.login(AK)


username = 'Blackoutta'
model_name = 'Qwen2.5-3B-Instruct-sft-nl2sql-lora'
model_id = username + "/" + model_name,

api.push_model(
    model_id=model_id[0], # 如果model_id对应的模型库不存在，将会被自动创建
    model_dir=best_model_output_dir # 指定本地模型所在目录
)

2024-12-27 09:55:15,468 - modelscope - WARNING - No configuration.json file found in /root/llm_adv_qa/resources/sft_models/nl2sql/best, creating a default one.
2024-12-27 09:55:15,560 - modelscope - INFO - Creating new model [Blackoutta/Qwen2.5-3B-Instruct-sft-nl2sql-lora]
2024-12-27 09:55:17,225 - modelscope - INFO - Pushing folder /root/llm_adv_qa/resources/sft_models/nl2sql/best as model Blackoutta/Qwen2.5-3B-Instruct-sft-nl2sql-lora.
2024-12-27 09:55:17,226 - modelscope - INFO - Total folder size 72.31 MB, this may take a while depending on actual pushing size...
2024-12-27 09:55:17,238 - modelscope - ERROR - Running git command: ['git', 'lfs', 'env'] failed 
 stdout:  
 stderr: git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log

2024-12-27 09:55:17,239 - modelscope - ERROR - git lfs is not installed, please install.
2024-12-27 09:55:17,822 - modelscope - ERROR - Running git command: ['git', 'lfs', 'env'] failed 
 stdout:  
 stderr: git: 'lfs' is n